In [8]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline



In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
image_gen = ImageDataGenerator(rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [26]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Activation

In [27]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9248)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,183,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,857 (4.66 MB)

 Trainable params: 1,221,857 (4.66 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
batch_size = 16
train_image_gen = image_gen.flow_from_directory('CATS_DOGS/train',
                                                target_size=(150, 150),
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 18743 images belonging to 2 classes.


In [30]:
test_image_gen = image_gen.flow_from_directory('CATS_DOGS/test',
                                                target_size=(150, 150),
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 6251 images belonging to 2 classes.


In [31]:
train_image_gen.class_indices

{'CAT': 0, 'DOG': 1}

In [38]:
results = model.fit(train_image_gen,epochs=50,steps_per_epoch=150,validation_data=test_image_gen)  

Epoch 1/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.4997 - loss: 0.6970

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 214ms/step - accuracy: 0.4998 - loss: 0.6970 - val_accuracy: 0.4986 - val_loss: 0.6921
Epoch 2/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 215ms/step - accuracy: 0.5344 - loss: 0.6950 - val_accuracy: 0.5964 - val_loss: 0.6763
Epoch 3/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 217ms/step - accuracy: 0.5667 - loss: 0.6844 - val_accuracy: 0.5212 - val_loss: 0.6893
Epoch 4/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 34s 229ms/step - accuracy: 0.5690 - loss: 0.6843 - val_accuracy: 0.5737 - val_loss: 0.6760
Epoch 5/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 214ms/step - accuracy: 0.5553 - loss: 0.6835 - val_accuracy: 0.5831 - val_loss: 0.6680
Epoch 6/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 221ms/step - accuracy: 0.6183 - loss: 0.6672 - val_accuracy: 0.5964 - val_loss: 0.6636
Epoch 7/50
131/150 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.5939 - loss: 0.6648

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


150/150 ━━━━━━━━━━━━━━━━━━━━ 34s 226ms/step - accuracy: 0.5954 - loss: 0.6643 - val_accuracy: 0.6039 - val_loss: 0.6566
Epoch 8/50
122/150 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.6228 - loss: 0.6544

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


150/150 ━━━━━━━━━━━━━━━━━━━━ 31s 208ms/step - accuracy: 0.6199 - loss: 0.6561 - val_accuracy: 0.6066 - val_loss: 0.6531
Epoch 9/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 34s 225ms/step - accuracy: 0.5750 - loss: 0.6644 - val_accuracy: 0.6121 - val_loss: 0.6613
Epoch 10/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 217ms/step - accuracy: 0.6102 - loss: 0.6555 - val_accuracy: 0.6391 - val_loss: 0.6406
Epoch 11/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 221ms/step - accuracy: 0.6576 - loss: 0.6394 - val_accuracy: 0.6453 - val_loss: 0.6317
Epoch 12/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 35s 231ms/step - accuracy: 0.6583 - loss: 0.6444 - val_accuracy: 0.6593 - val_loss: 0.6237
Epoch 13/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 37s 245ms/step - accuracy: 0.6111 - loss: 0.6500 - val_accuracy: 0.6537 - val_loss: 0.6199
Epoch 14/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 38s 253ms/step - accuracy: 0.6705 - loss: 0.6189 - val_accuracy: 0.6793 - val_loss: 0.5969
Epoch 15/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 37s 250ms/step - accuracy: 0.6604 - loss: 0.6253

In [37]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [40]:
results.history['accuracy'] = results.history.pop('accuracy', None)

In [43]:
results.history['accuracy']

[0.5083333253860474,
 0.5425000190734863,
 0.55708909034729,
 0.5770833492279053,
 0.5645833611488342,
 0.6037499904632568,
 0.6087499856948853,
 0.6070696711540222,
 0.6083333492279053,
 0.6108333468437195,
 0.6462500095367432,
 0.652864933013916,
 0.6179166436195374,
 0.6670833230018616,
 0.6570833325386047,
 0.6731557250022888,
 0.6650000214576721,
 0.6708333492279053,
 0.6883333325386047,
 0.6758333444595337,
 0.7012500166893005,
 0.7185277938842773,
 0.6949999928474426,
 0.7325819730758667,
 0.7034713625907898,
 0.7058333158493042,
 0.7129166722297668,
 0.7170833349227905,
 0.7133333086967468,
 0.715416669845581,
 0.7275000214576721,
 0.7264344096183777,
 0.7275000214576721,
 0.7266666889190674,
 0.7520833611488342,
 0.7354166507720947,
 0.7432036995887756,
 0.7350000143051147,
 0.7433333396911621,
 0.7325819730758667,
 0.7454166412353516,
 0.7599999904632568,
 0.7491666674613953,
 0.7641666531562805,
 0.7787500023841858,
 0.762499988079071,
 0.7720833420753479,
 0.758105993270874

In [44]:
from keras.models import load_model

In [45]:
new_model = load_model('cats_dogs50epochs.h5')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'cats_dogs50epochs.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [46]:
import pickle

In [47]:
filename = 'finalized_model.sav'

In [48]:
pickle.dump(model, open(filename, 'wb'))

In [49]:
model.save("catsdogs.h5")

In [50]:
new = load_model('catsdogs.h5')

In [52]:
from keras.preprocessing import image


dog = image.load_img('CATS_DOGS/test/DOG/9374.jpg', target_size=(150, 150))

In [53]:
dog = image.img_to_array(dog)

In [54]:
dog = np.expand_dims(dog, axis=0)

In [55]:
dog = dog/255

In [57]:
model.predict_classes(dog)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [59]:
pred = (model.predict(dog) > 0.5).astype("int32")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [60]:
pred

array([[1]], dtype=int32)

In [62]:
hashmap = {0: 'cat', 1: 'dog'}
list(hashmap.items())

[(0, 'cat'), (1, 'dog')]